![dpp](./img/examples.png)

# Flow Shop Scheduling Problem

--------------------
* Developed by _Keivan Tafakkori_
* Date: 18-4-2023
* Contact: https://www.linkedin.com/in/keivan-tafakkori/
* GitHub: https://github.com/ktafakkori
--------------------

### Packages

In [5]:
from feloopy import *

### Setting

In [6]:
problem_name = 'fssp'
solution_method = 'exact'
interface = 'pulp'
solver = 'highs'
key = 0

### Model

In [7]:
#Environment
m = target_model(solution_method, problem_name, interface, key=key)

#Dataset
w = [0.1, 0.4, 0.15, 0.35] #Priority weight of each job
p = [[  7,   3,    9,    4],
     [  7,   3,    9,    4]] #Processing time of each job on each machine
s = [5, 2] #Setup times of the machines


#Sets
I = range(len(p[0])) #Set of jobs
J = range(len(I)) #Set of positions
K = range(len(p)) #Set of machines

#Variables
x = m.bvar('x', [I,J])
c = m.pvar('c', [J,K])
d = m.pvar('d', [J,K])

#Objective
m.obj(sum(w[j]*c[j,1] for j in J))

#Constraints
for i in I: 
    m.con(sum(x[i,j] for j in J) == 1)

for j in J: 
    m.con(sum(x[i,j] for i in I) == 1)

for j in J: 
    if j!=0: 
        m.con(c[j,1] >= sum(x[i,j]*p[1][i] for i in I) + d[j,1])

m.con(c[0,1] == s[1] + sum(x[i,0]*p[1][i] for i in I) + c[0,0])

for j in J: 
    if j!=0: 
        m.con(c[j,0] >= c[j-1,0] + sum(x[i,j]*p[0][i] for i in I))

m.con(c[0,0] == s[0] + sum(x[i,0]*p[0][i] for i in I))

for j in J: 
    if j!=0: 
        m.con(d[j,1] >= c[j-1,1])

for j in J: 
    m.con(d[j,1] >= c[j,0])

#Solve
m.sol(['min'], solver)

### Result

In [8]:
#Report
m.report()


+--------------------------------------------------------------------------------+
|                                 FelooPy v0.2.5                                 |
+--------------------------------------------------------------------------------+
| Date: 2023-05-27                                                Time: 19:06:57 |
| Interface: pulp                                                  Solver: highs |
+--------------------------------------------------------------------------------+
|                               Model Information                                |
+--------------------------------------------------------------------------------+
|                             The 'fssp' model has:                              |
|                    16 positive variable(s) in 2 class(es).                     |
|                     16 binary variable(s) in 1 class(es).                      |
|                                1 objective(s).                                 |
|  